# Data_Augmentation_GANs

This repository is a deep learning project that brings together the Discriminator (the Truth Seeker) and the Generator (the Artful Creator) to create and evaluate beautiful, yet deceiving, images. Here, we present the code snippets that define the Discriminator and Generator networks, their training process, and the visual results.

In [ ]:
# Import necessary libraries
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Set the Dataset Directory and Display Information

This code sets the directory containing the dataset of images, checks the number of images in the dataset, and displays the names of the first ten images.

In [ ]:
# Set the directory containing the dataset
Image_directories = '/content/drive/My Drive/celebrities-100k/100k/'
print(f"Number of images in the dataset: {len(os.listdir(DATA_DIR+'/100k'))}")

In [ ]:
# Display the first 10 files in the dataset directory
print(f"Sample of the dataset files: {os.listdir(Image_directories+'/100k')[:10]}")

### Define Image Size, Batch Size, and Normalization Statistics:
It establishes the desired image size, batch size for training, and defines the statistical values (mean and standard deviation) for image normalization.

In [ ]:
# Define image size, batch size, and image statistics for normalization
image_size = 64
batch_size = 128
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)  # Mean and standard deviation for image normalization

# Create a training dataset with transformations
train_ds = ImageFolder(root=Image_directories,
                       transform=T.Compose([T.Resize(image_size),
                                            T.CenterCrop(image_size),  # Crop the center square of the image
                                            T.ToTensor(),
                                            T.Normalize(*stats)  # Normalize images to the range -1 to 1
                                        ]))





### Create Training Dataset and Data Loader:
 This code creates a training dataset with image transformations like resizing and center cropping and sets up a data loader to efficiently load the data in batches for training.


In [ ]:
# Create a data loader for training
train_data = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)  # Utilize multiple CPU cores

### Define Denormalization and Display Images:

It defines functions for denormalizing image tensors and displaying a batch of images in a grid for visualization.

In [ ]:
# Define a function to denormalize image tensors
def denormalization(img_tensors):
    "Denormalize image tensor with specified mean and std"
    return img_tensors * stats[1][0] + stats[0][0]

# Define a function to display a batch of images
def display_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(denormalization(images.detach()[:nmax]), nrow=8).permute(1, 2, 0))

### Sneak Peek:
A quick look at the generated images by the Generator even before training begins.

In [ ]:
# Display a batch of images from the training dataset
display_images(train_data)

### Define Device Handling Functions:

These functions handle device selection (GPU or CPU) and enable the transfer of data and models to the chosen device, ensuring efficient computation on available hardware.

In [ ]:
# Define a function to get the default device (GPU if available, else CPU)
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

# Define a function to move tensor(s) to the chosen device
def to_device(data, device):
    """Transfer tensor(s) to the specified device."""
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

# Create a class to wrap a dataloader and move data to a device
class DeviceDataLoader():
    """Encapsulate a dataloader while transferring data to a device."""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to the device."""
        for batch in self.dl:
            yield to_device(batch, self.device)

    def __len__(self):
        """Return the number of batches in the dataloader."""
        return len(self.dl)

device = get_default_device()


In [ ]:
train_dl = DeviceDataLoader(train_dl, device)

## Let's build the GAN model


### Meet the Discriminator:

The code introduces the architecture of the Discriminator network, which critically evaluates generated images.

The Discriminator is a neural network designed for distinguishing real images from fake ones. It comprises multiple convolutional layers with batch normalization and LeakyReLU activation functions, followed by a binary classification layer that outputs a single value (1 or 0) to determine if the input image is real or generated.



In [ ]:
discriminator = nn.Sequential(
    # in: 3x 64 x 64
    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 64 x 32 x 32

    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 128 x 16 x 16

    nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 256 x 8 x 8

    nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 512 x 4 x 4

    nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
    # out: 1 x 1 x 1

    nn.Flatten(),
    nn.Sigmoid()
)

In [ ]:
discriminator = to_device(discriminator, device)

In [ ]:
# create a tensor Batch_Size,C,H,W
X = torch.rand(size=(1, 3, 64, 64), dtype=torch.float32, device=device)
for layer in discriminator:
    X = layer(X)
    print(layer.__class__.__name__,'output shape: \t',X.shape)

### Meet the Generator:
The code reveals the Generator's architecture, responsible for creating beautiful artwork.

The Generator, on the other hand, is a neural network responsible for generating fake images that aim to fool the Discriminator. It starts with a latent vector and utilizes transposed convolutional layers with batch normalization and ReLU activation functions to progressively upscale and create realistic-looking images.



In [ ]:
latent_size = 128

In [ ]:
generator = nn.Sequential(
    # in: latent_size x 1 x 1

    nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    # out: 512 x 4 x 4

    nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    # out: 256 x 8 x 8

    nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    # out: 128 x 16 x 16

    nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    # out: 64 x 32 x 32

    nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh()  # output is between -1 to 1
    # out: 3 x 64 x 64
)

In [ ]:
X = torch.randn(size=(1, 128, 1, 1))
for layer in generator:
  X = layer(X)
  print(layer.__class__.__name__,'output shape: \t',X.shape)

In [ ]:
xb = torch.randn(batch_size, latent_size, 1, 1) # random latent tensors
fake_images = generator(xb)
print(fake_images.shape)
show_images(fake_images)

In [ ]:
generator = to_device(generator, device) # move generator to device



## The Artful Journey
- Training the Generator: Learn how the Artful Creator refines its skills to generate images that "fool" the Discriminator.
- The Training Loop: Discover the process where art meets truth, with both the Discriminator and Generator honing their abilities.

Training the Discriminator involves two key steps:
1. Evaluating real images by computing their loss and target labels (1 for real), and
2. Generating fake images, calculating their loss and target labels (0 for fake), and optimizing the Discriminator's weights to enhance its ability to distinguish real from fake.



In [ ]:
def train_discriminator(real_images, opt_d):
  # Clear discriminator gradients
  opt_d.zero_grad()

  # Pass real images through  discriminator
  real_preds = discriminator(real_images)
  real_targets = torch.ones(real_images.size(0), 1, device=device)
  real_loss = F.binary_cross_entropy(real_preds, real_targets)
  real_score = torch.mean(real_preds).item()

  # Generate fake images
  latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
  fake_images = generator(latent)

  # Pass Fake images through discriminator
  fake_targets = torch.zeros(fake_images.size(0), 1, device=device)
  fake_preds = discriminator(fake_images)
  fake_loss = F.binary_cross_entropy(fake_preds, fake_targets)
  fake_score = torch.mean(fake_preds).item()

  # Update discriminator weights
  loss = real_loss + fake_loss
  loss.backward()
  opt_d.step()
  return loss.item(), real_score, fake_score

Training the Generator is about generating fake images, attempting to deceive the Discriminator (by assigning target labels of 1 for real), and updating the Generator's weights to improve its capability to generate more convincing images.



In [ ]:
def train_generator(opt_g):
  # Clear generator gradients
  opt_g.zero_grad()

  # Generate fake images
  latent = torch.randn(batch_size, latent_size, 1,1, device=device)
  fake_images = generator(latent)

  # Try to fool the discriminator
  preds = discriminator(fake_images)
  targets = torch.ones(batch_size, 1, device=device)
  loss = F.binary_cross_entropy(preds, targets)

  # Update generator
  loss.backward()
  opt_g.step()

  return loss.item()

In the process, both the Discriminator and Generator iteratively adjust their weights to reach a balance where the Generator creates increasingly realistic images and the Discriminator gets better at distinguishing real from fake images. This adversarial training leads to the generation of high-quality images by the Generator.


## Saving Masterpieces
- The code provides functionality to save the masterpieces generated by the Generator during training, allowing you to witness the artistic evolution.


In [ ]:
from torchvision.utils import save_image


In [ ]:
sample_dir = 'augmented'
os.makedirs(sample_dir, exist_ok=True)

In [ ]:
def save_samples(index, latent_tensors, show=True):
  fake_images = generator(latent_tensors)
  fake_fname = 'augmented=images-{0:0=4d}.png'.format(index)
  save_image(denormalization(fake_images), os.path.join(sample_dir, fake_fname), nrow=8)
  print("Saving", fake_fname)

  if show:
    fig, ax = plt.subplots(figsize=(8,8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1, 2, 0))

In [ ]:
fixed_latent = torch.randn(64, latent_size, 1, 1, device=device)


In [ ]:
save_samples(0, fixed_latent)

In [ ]:
from tqdm.notebook import tqdm
import torch.nn.functional as F

In [ ]:
def fit(epochs, lr, start_idx = 1):
  torch.cuda.empty_cache()

  # Losses & scores
  losses_g = []
  losses_d = []
  real_scores = []
  fake_scores = []

  # Create optimizers
  opt_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
  opt_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))

  for epoch in range(epochs):
    for real_images, _ in tqdm(train_dl):
      # Train discriminator
      loss_d, real_score, fake_score = train_discriminator(real_images, opt_d)
      # Train generator
      loss_g = train_generator(opt_g)

    # Record losses & scores
    losses_g.append(loss_g)
    losses_d.append(loss_d)
    real_scores.append(real_score)
    fake_scores.append(fake_score)

    # Log losses & scores (last batch)
    print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(epoch+1, epochs, loss_g, loss_d, real_score, fake_score))
    # Save generated images
    save_samples(epoch+start_idx, fixed_latent, show=False)

  return losses_g, losses_d, real_scores, fake_scores

In [ ]:
# Hyperparameters
lr = 0.00025
epochs = 60

In [ ]:
history = fit(epochs, lr)

In [ ]:
# Save the model checkpoints
torch.save(generator.state_dict(), 'G.pth')
torch.save(discriminator.state_dict(), 'D.pth')

In [ ]:
losses_g, losses_d, real_scores, fake_scores = history


## Visualization and Export
- The code shows how to visualize losses and scores during training, helping you understand the training process.
- Export the generated images and even create a video to visualize the progress of the model.

In [ ]:
from IPython.display import Image


In [ ]:
Image('./augmented/augmented=images-0001.png')

In [ ]:
Image('./augmented/augmented=images-0060.png')

In [ ]:
import cv2
import os

vid_fname = 'gans_training.avi'

print("Starting converting images to video.")
files = [os.path.join(sample_dir, f) for f in os.listdir(sample_dir) if 'augmented' in f]
files.sort()

print(files)

fourcc = cv2.VideoWriter_fourcc(*'MPEG')
out = cv2.VideoWriter(vid_fname,fourcc, 1.0, (640,480))
[out.write(cv2.imread(fname)) for fname in files]
out.release()
print("DONE!")

In [ ]:
plt.plot(losses_d, '-')
plt.plot(losses_g, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');

In [ ]:
plt.plot(real_scores, '-')
plt.plot(fake_scores, '-')
plt.xlabel('epoch')
plt.ylabel('score')
plt.legend(['Real', 'Fake'])
plt.title('Scores');